In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 在GCP上进行端到端的机器学习：MLOps阶段6：使用FastAPI和Vertex AI Prediction开始

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage6/get_started_with_fastapi.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage6/get_started_with_fastapi.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/ml_ops/stage6/get_started_with_fastapi.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>        
</table>
<br/><br/><br/>

## 概述

本教程演示了如何使用使用`FastAPI`构建的自定义服务二进制文件从`Vertex AI Endpoint`中提供预测。

### 目标

在本教程中，您将学习如何使用`Vertex AI Prediction`在`Vertex AI Endpoint`上使用`FastAPI`自定义服务二进制文件。

本教程使用以下谷歌云ML服务和资源：

- `Vertex AI Prediction`
- `Vertex AI Models`
- `Vertex AI Endpoints`

执行的步骤包括：

- 从TensorFlow Hub下载预训练的图像分类模型。
- 创建一个用于接收压缩图像数据并输出解压缩后的预处理数据供模型输入的服务函数。
- 将TensorFlow Hub模型和服务函数上传为`Vertex AI Model`资源。
- 创建一个`Endpoint`资源。
- 使用`FastAPI`自定义服务二进制部署`Model`资源到`Endpoint`资源。
- 对部署到`Endpoint`资源的`Model`资源实例进行在线预测。

数据集

此教程使用了来自TensorFlow Hub的预训练图像分类模型，该模型是在ImageNet数据集上训练的。

了解更多关于[ResNet V2预训练模型](https://tfhub.dev/google/imagenet/resnet_v2_101/classification/5)。

### 费用

本教程使用 Google Cloud 的计费组件：

* Vertex AI
* Cloud Storage

了解 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage 价格](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据您的预期使用情况生成费用估算。

## 安装

安装以下软件包以执行该笔记本。

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG -q
! pip3 install --upgrade google-cloud-pipeline-components $USER_FLAG -q
! pip3 install tensorflow-hub $USER_FLAG -q

### 重新启动内核

安装完额外的包之后，您需要重新启动笔记本内核，以便它能找到这些包。

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 开始之前

### GPU运行时

*如果可以的话，请确保在GPU运行时中运行此笔记本。在Colab中，选择* **运行时 > 更改运行时类型 > GPU**

### 设置您的谷歌云项目

**以下步骤是必需的，无论您的笔记本环境如何。**

1. [选择或创建谷歌云项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建帐户时，您将获得300美元的免费信用额度来支付计算/存储成本。

2. [确保为您的项目启用了结算。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用以下API：Vertex AI APIs，Compute Engine APIs和Cloud Storage。](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component,storage-component.googleapis.com)

4. 如果您在本地运行此笔记本，您需要安装[Cloud SDK](https://cloud.google.com/sdk)。

5. 在下面的单元格中输入您的项目ID。然后运行该单元格，以确保Cloud SDK在此笔记本中的所有命令中使用正确的项目。

**注意**：Jupyter会将以`!`为前缀的行视为shell命令，并对以`$`为前缀的Python变量进行插值。

设置您的项目ID

**如果您不知道您的项目ID**，您可以使用`gcloud`获取您的项目ID。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 区域

您还可以更改用于此笔记本其余部分操作的 `REGION` 变量。以下是 Vertex AI 支持的区域。我们建议您选择距离您最近的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太：`asia-east1`

您可能不能使用多区域存储桶来训练 Vertex AI。并非所有区域都支持所有 Vertex AI 服务。

了解更多有关 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

时间戳

如果您正在进行实时教程会话，您可能正在使用一个共享的测试账户或项目。为了避免用户之间在创建资源时发生名称冲突，您为每个实例会话创建一个时间戳，并将时间戳附加到您在本教程中创建的资源的名称中。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 验证您的 Google Cloud 帐户

**如果您正在使用 Vertex AI Workbench Notebooks**，您的环境已经通过验证。请跳过这一步。

**如果您正在使用 Colab**，运行下面的单元格，并按照提示进行身份验证以通过 oAuth 认证您的帐户。

**否则**，请按照以下步骤操作：

在 Cloud Console 中，转到 [创建服务帐号密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey) 页面。

**单击创建服务帐号**。

在**服务帐号名称**字段中输入一个名称，并单击**创建**。

在**将此服务帐号授予项目访问权限**部分，单击角色下拉列表。在过滤框中键入 "Vertex"，并选择**Vertex 管理员**。在过滤框中键入 "Storage Object Admin"，并选择**存储对象管理员**。

单击创建。包含您密钥的 JSON 文件将下载到您的本地环境。

在下面的单元格中，将您的服务帐号密钥路径输入为 GOOGLE_APPLICATION_CREDENTIALS 变量，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = False
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        IS_COLAB = True
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您使用的是哪种笔记本环境，都需要按照以下步骤进行。**

当您为 Python 初始化 Vertex AI SDK 时，您需要指定一个云存储暂存桶。这个暂存桶是您的数据集和模型资源在会话之间保留的地方。

请在下面设置您的云存储桶的名称。桶的名称必须在所有谷歌云项目中全局唯一，包括您组织外的项目。

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_URI == "" or BUCKET_URI is None or BUCKET_URI == "gs://[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + TIMESTAMP
    BUCKET_URI = "gs://" + BUCKET_NAME

只有当您的存储桶还不存在时：运行以下单元格以创建您的云存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

最后，通过检查云存储桶的内容来验证访问权限。

In [ ]:
! gsutil ls -al $BUCKET_URI

### 设置变量

接下来，设置一些在教程中使用的变量。

### 导入库并定义常量

In [ ]:
import google.cloud.aiplatform as aip
import tensorflow as tf
import tensorflow_hub as hub

### 初始化用于Python的Vertex AI SDK

为您的项目和对应的存储桶初始化用于Python的Vertex AI SDK。

In [ ]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

设置硬件加速器

您可以为训练和预测设置硬件加速器。

设置变量`DEPLOY_GPU/DEPLOY_NGPU`以使用支持GPU的容器映像，并指定分配给虚拟机实例的GPU数量。例如，要使用一个支持GPU的容器映像，并且为每个虚拟机分配4个Nvidia Telsa K80 GPU，则需要指定：

    (aip.AcceleratorType.NVIDIA_TESLA_K80, 4)

否则，指定`(None, None)`以使用一个在CPU上运行的容器映像。

了解更多关于[您位置的硬件加速器支持的信息](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators)。

*注意*：在2.3之前的TF版本中，GPU支持将无法在本教程中加载自定义模型。这是一个已知问题，在TF 2.3中已修复。这是由于在服务函数中生成的静态图操作造成的。如果在您自己的自定义模型上遇到此问题，请使用带有GPU支持的TF 2.3容器映像。

In [ ]:
if os.getenv("IS_TESTING_DEPLOY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aip.gapic.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_DEPLOY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (None, None)

#### 设置机器类型

接下来，设置用于预测的机器类型。

- 将变量`DEPLOY_COMPUTE`设置为配置用于预测的虚拟机的计算资源。
 - `机器类型`
     - `n1-standard`：每个vCPU 3.75GB内存。
     - `n1-highmem`：每个vCPU 6.5GB内存
     - `n1-highcpu`：每个vCPU 0.9GB内存
 - `vCPUs`：数量为\[2, 4, 8, 16, 32, 64, 96 \]

*注意：您也可以使用n2和e2机器类型进行训练和部署，但它们不支持GPU。*

In [ ]:
if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", DEPLOY_COMPUTE)

### 启用Artifact Registry API

您必须为您的项目启用Artifact Registry API服务。

了解更多关于[启用服务](https://cloud.google.com/artifact-registry/docs/enable-service)。

In [ ]:
! gcloud services enable artifactregistry.googleapis.com

## 创建一个私有的 Docker 仓库

您的第一步是在 Google Artifact Registry 中创建您自己的 Docker 仓库。

1. 运行 `gcloud artifacts repositories create` 命令，在您的区域内创建一个新的 Docker 仓库，描述为 "docker 仓库"。

2. 运行 `gcloud artifacts repositories list` 命令，验证您的仓库是否已创建。

In [ ]:
PRIVATE_REPO = "my-docker-repo"

! gcloud artifacts repositories create {PRIVATE_REPO} --repository-format=docker --location={REGION} --description="Docker repository"

! gcloud artifacts repositories list

### 配置身份验证到您的私有仓库

在推送或拉取容器镜像之前，配置Docker以使用`gcloud`命令行工具来验证请求到您地区的`Artifact Registry`。

In [ ]:
! gcloud auth configure-docker {REGION}-docker.pkg.dev --quiet

## 从 TensorFlow Hub 获取预训练模型

为了演示目的，本教程使用了从 TensorFlow Hub(TFHub) 获取的预训练模型，然后将其上传到 `Vertex AI Model` 资源。一旦您有了一个 `Vertex AI Model` 资源，该模型就可以部署到一个 `Vertex AI Endpoint` 资源中。

### 下载预训练模型

首先，您需要从 TensorFlow Hub 下载预训练模型。该模型将作为一个 TF.Keras 层进行下载。在这个示例中，为了完成模型的构建，您将创建一个带有下载的 TFHub 模型作为一层的 `Sequential()` 模型，并指定模型的输入形状。

In [ ]:
tfhub_model = tf.keras.Sequential(
    [hub.KerasLayer("https://tfhub.dev/google/imagenet/resnet_v2_101/classification/5")]
)

tfhub_model.build([None, 224, 224, 3])

tfhub_model.summary()

###保存模型工件

此时，模型位于内存中。接下来，您将模型工件保存到云存储位置。

In [ ]:
MODEL_DIR = BUCKET_URI + "/model"
tfhub_model.save(MODEL_DIR)

## 上传模型以提供服务

接下来，您将从自定义作业上传您的 TF.Keras 模型到 Vertex `Model` 服务，这将为您的自定义模型创建一个 Vertex `Model` 资源。在上传过程中，您需要定义一个服务函数，将数据转换为模型期望的格式。如果您将编码数据发送给 Vertex AI，您的服务函数将确保在将数据作为输入传递给模型之前，在模型服务器上对数据进行解码。

### 服务函数如何工作

当您向在线预测服务器发送请求时，请求会被一个 HTTP 服务器接收。HTTP 服务器会从 HTTP 请求内容体中提取预测请求。提取的预测请求会被转发给服务函数。对于 Google 预构建的预测容器，请求内容会以 `tf.string` 的形式传递给服务函数。

服务函数包括两个部分：

- `预处理函数`：
  - 将输入 (`tf.string`) 转换为底层模型的输入形状和数据类型（动态图）。
  - 执行与训练底层模型期间相同的数据预处理 -- 例如，标准化，缩放等。
- `后处理函数`:
  - 将模型输出转换为接收应用程序所期望的格式 -- 如，压缩输出。
  - 为接收应用程序打包输出 -- 例如，添加标题，构建 JSON 对象等。

预处理和后处理函数都会转换为静态图，然后与模型融合。底层模型的输出会被传递给后处理函数。后处理函数会将转换/打包后的输出传递回HTTP服务器。HTTP 服务器会将输出作为 HTTP 响应内容返回。

在为 TF.Keras 模型构建服务函数时，您需要考虑一个因素，即它们运行为静态图。这意味着您不能使用需要动态图的 TF 图操作。如果您这样做，您将在服务函数的编译过程中收到错误提示，指出您正在使用不受支持的 EagerTensor。

### 图像数据的服务功能

#### 预处理

为了将图像传递给预测服务，您需要将经过压缩的（例如JPEG）图像字节编码为Base 64 — 这样可以在通过网络传输二进制数据时保证内容不被修改。由于部署的模型期望输入数据为原始（未压缩）字节，您需要确保将Base 64编码的数据转换回原始字节，然后对其进行预处理以匹配模型输入的要求，然后将其作为输入传递给部署的模型。

为了解决这个问题，您需要定义一个服务函数（`serving_fn`）并将其附加到模型作为预处理步骤。添加一个`@tf.function`装饰器，这样服务函数就会与底层模型融合在一起（而不是在CPU上游运行）。

当您发送预测或解释请求时，请求的内容会被解码为一个Tensorflow字符串（`tf.string`），然后传递给服务函数（`serving_fn`）。服务函数会将`tf.string`预处理成原始（未压缩）的numpy字节（`preprocess_fn`），以匹配模型的输入要求：

- `io.decode_jpeg` - 解压缩JPG图像，返回一个具有三个通道（RGB）的Tensorflow张量。
- `image.convert_image_dtype` - 将整数像素值转换为浮点32，并重新调整像素数据在0到1之间。
- `image.resize` - 调整图像大小以匹配模型的输入形状。

在这一点上，数据可以通过一个具体函数传递到模型（`m_call`）。服务函数是一个静态图，而模型是一个动态图。具体函数执行将输入数据从服务函数传递到模型的任务，并将模型的预测结果从模型传递回服务函数的任务。

In [ ]:
CONCRETE_INPUT = "numpy_inputs"


def _preprocess(bytes_input):
    decoded = tf.io.decode_jpeg(bytes_input, channels=3)
    decoded = tf.image.convert_image_dtype(decoded, tf.float32)
    resized = tf.image.resize(decoded, size=(224, 224))
    return resized


@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def preprocess_fn(bytes_inputs):
    decoded_images = tf.map_fn(
        _preprocess, bytes_inputs, dtype=tf.float32, back_prop=False
    )
    return {
        CONCRETE_INPUT: decoded_images
    }  # User needs to make sure the key matches model's input


@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def serving_fn(bytes_inputs):
    images = preprocess_fn(bytes_inputs)
    prob = m_call(**images)
    return prob


m_call = tf.function(tfhub_model.call).get_concrete_function(
    [tf.TensorSpec(shape=[None, 224, 224, 3], dtype=tf.float32, name=CONCRETE_INPUT)]
)

tf.saved_model.save(tfhub_model, MODEL_DIR, signatures={"serving_default": serving_fn})

获取服务函数签名

您可以通过重新加载模型到内存中，并查询每个层对应的签名来获取模型的输入和输出层的签名。

为了您的目的，您需要服务函数的签名。为什么？因为当我们将数据作为HTTP请求包发送进行预测时，图像数据是base64编码的，而我们的TF.Keras模型需要numpy输入。您的服务函数将会将数据从base64转换为一个numpy数组。

在发出预测请求时，您需要将请求路由到服务函数而不是模型，因此您需要知道服务函数的输入层名称 -- 这将在您进行预测请求时使用。

In [ ]:
loaded = tf.saved_model.load(MODEL_DIR)

serving_input = list(
    loaded.signatures["serving_default"].structured_input_signature[1].keys()
)[0]
print("Serving function input:", serving_input)

使用FastAPI构建一个HTTP服务器

您的自定义容器映像要求容器必须运行一个HTTP服务器。容器必须监听并响应活跃性检查、健康检查和预测请求。

在本教程中，您将使用FastAPI来实现HTTP服务器。HTTP服务器必须在0.0.0.0上监听请求。

了解更多关于[FastAPI](https://fastapi.tiangolo.com/)的信息。

In [ ]:
# Make folder for Python training script
! rm -rf serve
! mkdir serve

# Make the predictor subfolder
! mkdir serve/app

为服务器创建要求文件

接下来为服务器环境创建`requirements.txt`文件，该文件指定了需要在服务容器上安装的Python包。

In [ ]:
%%writefile serve/requirements.txt

numpy~=1.20
tensorflow>=2.5
google-cloud-storage>=1.26.0,<2.0.0dev

### 编写 FastAPI 服务脚本

接下来，您可以使用 `FastAPI` 编写 HTTP 服务器的服务脚本，具体如下：

- `app`：实例化一个 `FastAPI` 应用程序
- `health()`：定义对健康请求的响应。
    - 返回状态码 200
- `predict()`：定义对预测请求的响应。
    - `body = await request.json()`：异步等待 HTTP 请求。
    - `instances = body["instances"]`：预测请求的内容。
    - `outputs = model.predict(inputs)`：调用模型进行预测。
    - `return {"predictions": ... }`：在响应体中返回格式化的预测结果。

In [ ]:
%%writefile serve/app/main.py
from fastapi.logger import logger
from fastapi import FastAPI, Request
import tensorflow as tf

import numpy as np
import argparse
import os

import logging

gunicorn_logger = logging.getLogger('gunicorn.error')
logger.handlers = gunicorn_logger.handlers

if __name__ != "main":
    logger.setLevel(gunicorn_logger.level)
else:
    logger.setLevel(logging.DEBUG)

app = FastAPI()

logger.info("Loading model")
model = tf.saved_model.load('/model')

@app.get(os.environ['AIP_HEALTH_ROUTE'], status_code=200)
def health():
    """ health check to ensure HTTP server is ready to handle 
        prediction requests
    """
    return {"status": "healthy"}


@app.post(os.environ['AIP_PREDICT_ROUTE'])
async def predict(request: Request):
    body = await request.json()
    instances = body["instances"]
    inputs = []
    for instance in instances:
        inputs.append(instance['bytes_inputs']['b64'])
       
    # unfinished, returns Internal Server error
    #outputs = model.predict(inputs)
    #logger.info(f"Outputs {outputs}")
    #return {"predictions": [class_num for class_num in np.argmax(outputs, axis=1)]}
    return {'test': 'to-be-finished'}

### 添加预启动脚本

FastAPI将在启动服务器之前执行此脚本。为了在与Vertex AI期望的端口相同的端口上运行FastAPI，将PORT环境变量设置为等于`AIP_HTTP_PORT`。

In [ ]:
%%writefile serve/app/prestart.sh

#!/bin/bash
export PORT=$AIP_HTTP_PORT

创建Dockerfile，使用tiangolo/uvicorn-gunicorn-fastapi作为基础镜像。这将自动使用Gunicorn和Uvicorn为您运行FastAPI。

In [ ]:
%%bash -s $MODEL_DIR 

MODEL_DIR=$1

mkdir -p ./serve/model/
gsutil cp -r ${MODEL_DIR} ./serve/model/ 

cat > ./serve/Dockerfile <<EOF
FROM tiangolo/uvicorn-gunicorn-fastapi:python3.7

COPY ./app /app
COPY ./model /
COPY requirements.txt requirements.txt

RUN pip install -r requirements.txt

EXPOSE 7080
CMD ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "7080"]

EOF

#### 用于提供服务的容器（Docker）镜像

设置用于提供预测的FastAPI Serving Docker容器镜像。

1. 从Docker Hub拉取相应的用于TF Serving的CPU或GPU Docker镜像。
2. 为图像创建一个标记，用于在Artifact Registry中注册图像。
3. 使用Artifact Registry注册图像。

了解更多关于[使用Docker部署FastAPI](https://fastapi.tiangolo.com/deployment/docker/)。

In [ ]:
SUDO = ""  # "sudo"

DEPLOY_IMAGE = (
    f"{REGION}-docker.pkg.dev/" + PROJECT_ID + f"/{PRIVATE_REPO}" + "/fastapi"
)

! {SUDO} docker build serve -t $DEPLOY_IMAGE
! {SUDO} docker push $DEPLOY_IMAGE

print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

### 在本地运行和测试容器（可选）

以分离模式在本地运行容器并提供容器所需的环境变量。这些环境变量将在部署后由Vertex AI Prediction服务提供给容器。测试/health和/predict路径，然后停止正在运行的镜像。

在将容器镜像推送到Artifact Registry以与Vertex AI Predictions一起使用之前，您可以在本地环境中将其作为容器运行，以验证服务器是否按预期工作。

要在本地将容器镜像作为容器运行，请运行以下命令：

In [ ]:
! {SUDO} docker rm local-fastapi 2>/dev/null
! {SUDO} docker run -t -d --rm -p 7080:7080 \
    --name=local-fastapi \
    -e AIP_HTTP_PORT=7080 \
    -e AIP_HEALTH_ROUTE=/health \
    -e AIP_PREDICT_ROUTE=/predict \
    -e AIP_STORAGE_URI={MODEL_DIR} \
    -e AIP_MODEL_DIR={MODEL_DIR} \
    {DEPLOY_IMAGE}
! {SUDO} docker container ls
! sleep 10

要发送容器服务器的健康检查，请运行以下命令。输出应为{"status": "healthy"}。

In [ ]:
! curl http://localhost:7080/health

### 为预测准备测试数据

接下来，您将把一个压缩的JPEG图片加载到内存中，然后进行base64编码。为了演示目的，您将使用来自鲜花数据集的一张图片。

In [ ]:
! gsutil cp gs://cloud-ml-data/img/flower_photos/daisy/100080576_f52e8ee070_n.jpg test.jpg

In [ ]:
import base64

with open("test.jpg", "rb") as f:
    data = f.read()
b64str = base64.b64encode(data).decode("utf-8")

### 进行本地预测

接下来，通过进行本地预测请求来测试容器。

**待办事项**：请注意，服务应用程序中的预测路由是不完整的，并且只返回Base64编码的字符串，而不会调用模型。请查看代码中的注释。

In [ ]:
import json

instance = {serving_input: {"b64": b64str}}
instances = {"instances": [instance]}
s = json.dumps(instances)
with open("serve/instances.json", "w") as f:
    f.write(s)

In [ ]:
! curl -X POST \
  -d @serve/instances.json \
  -H "Content-Type: application/json; charset=utf-8" \
  http://localhost:7080/predict

要停止容器，运行以下命令：

In [ ]:
! {SUDO} docker stop local-fastapi

### 将TensorFlow Hub模型上传到`Vertex AI模型`资源

最后，您将TFHub模型和服务功能中的模型工件上传到`Vertex AI模型`资源中。由于您正在使用非Google预构建的服务二进制文件 - 例如TensorFlow Serving，您需要指定以下额外的服务配置设置：

- `serving_container_health_route`：服务用于定期ping以验证服务二进制文件正在运行的URL。
- `serving_container_predict_route`：用于将基于REST的预测请求路由到服务的URL。
- `serving_container_ports`：HTTP服务器用于监听请求的端口列表。

将模型上传到Vertex AI模型资源将返回一个长时间运行的操作，因为这可能需要一些时间。

*注意：*当您将模型工件上传到`Vertex AI模型`资源时，您需要指定相应的部署容器镜像。

In [ ]:
MODEL_NAME = "example_" + TIMESTAMP

model = aip.Model.upload(
    display_name="example_" + TIMESTAMP,
    artifact_uri=MODEL_DIR,
    serving_container_image_uri=DEPLOY_IMAGE,
    serving_container_health_route="/ping",
    serving_container_predict_route="/predict",
    serving_container_ports=[7080],
)

print(model)

创建`Endpoint`资源

您可以使用`Endpoint.create()`方法创建`Endpoint`资源。至少，您需要为端点指定显示名称。您还可以选择指定项目和位置（区域）；否则设置将继承您使用`init()`方法初始化Vertex AI SDK时设置的值。

在此示例中，指定了以下参数：

- `display_name`：`Endpoint`资源的人类可读名称。
- `project`：您的项目ID。
- `location`：您的区域。
- `labels`：（可选）`Endpoint`的用户定义的元数据，以键/值对的形式。

此方法返回一个`Endpoint`对象。

了解更多关于[Vertex AI Endpoints](https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api)。

In [ ]:
endpoint = aip.Endpoint.create(
    display_name="example_" + TIMESTAMP,
    project=PROJECT_ID,
    location=REGION,
    labels={"your_key": "your_value"},
)

print(endpoint)

## 部署 `Model` 资源到一个 `Endpoint` 资源。

您可以将一个或多个 `Vertex AI Model` 资源实例部署到同一个 Endpoint 中。每个部署的 `Vertex AI Model` 资源都将拥有自己的用于提供二进制文件的部署容器。

*注意:* 在之前的步骤中，您已经为 TFHub 模型指定了部署容器，讲上传模型工件到 `Vertex AI Model` 资源。

在下一个示例中，您将部署 `Vertex AI Model` 资源到一个 `Vertex AI Endpoint` 资源。`Vertex AI Model` 资源已经为它定义了部署容器镜像。要部署，您需要指定以下额外的配置设置:

- 机器类型。
- （如果有）GPU 的类型和数量。
- VM 实例的静态、手动或自动缩放。

在此示例中，您将使用最少量的指定参数来部署模型，如下所示:

- `model`: `Model` 资源。
- `deployed_model_displayed_name`: 部署模型实例的可读名称。
- `machine_type`: 每个 VM 实例的机器类型。

根据资源供应需求，此过程可能需要几分钟。

In [ ]:
response = endpoint.deploy(
    model=model,
    deployed_model_display_name="example_" + TIMESTAMP,
    machine_type=DEPLOY_COMPUTE,
)

print(endpoint)

### 进行预测

现在您的`Model`资源已部署到`Endpoint`资源，您可以通过向Endpoint资源发送预测请求来进行在线预测。

#### 请求

在这个例子中，由于您的测试项目位于云存储桶中，您可以使用`tf.io.gfile.Gfile()`打开并读取图像的内容。为了将测试数据传递给预测服务，您需要将字节编码为base64 -- 这样在通过网络传输二进制数据时内容就不会被修改。

每个实例的格式为：

    { serving_input: { 'b64': base64编码的字节 } }

由于`predict()`方法可以接受多个项目（实例），请将您的单个测试项目作为一个测试项目的列表发送。

#### 响应

从`predict()`调用的响应是一个Python字典，包含以下条目：

- `ids`：每个预测请求的内部分配的唯一标识符。
- `predictions`：每个类别标签的预测置信度，介于0到1之间。
- `deployed_model_id`：执行预测的部署的`Model`资源的Vertex AI标识符。

In [ ]:
# The format of each instance should conform to the deployed model's prediction input schema.
instances = [{serving_input: {"b64": b64str}}]

prediction = endpoint.predict(instances=instances)

print(prediction)

清理工作

要清理此项目中使用的所有Google Cloud资源，您可以删除用于此教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在此教程中创建的各个资源。

In [ ]:
! rm -rf serve test.jpg

delete_bucket = False
delete_model = True
delete_endpoint = True

if delete_endpoint:
    try:
        endpoint.undeploy_all()
        endpoint.delete()
    except Exception as e:
        print(e)

if delete_model:
    try:
        model.delete()
    except Exception as e:
        print(e)

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -rf {BUCKET_URI}